# Additional sampling (with dead zones)

In [1]:
import numpy as np
from scipy.stats import qmc

In [7]:
# Parameter ranges
activation_current = [600, 2000]
activation_duration = [600, 86400]
already_used = np.array([72629, 2512, 37919, 61969, 56564, 28106])
num_samples = 6

# Width of dead zones around each used value (± seconds)
dead_zone_width = 3600

# Step 1: Compute allowed zones
dead_zones = np.array([[x - dead_zone_width, x + dead_zone_width] for x in already_used])
dead_zones = np.clip(dead_zones, activation_duration[0], activation_duration[1])
dead_zones = dead_zones[np.argsort(dead_zones[:, 0])]  # sort by start

allowed_zones = []
start = activation_duration[0]
for low, high in dead_zones:
    if low > start:
        allowed_zones.append([start, low])
    start = max(start, high)
if start < activation_duration[1]:
    allowed_zones.append([start, activation_duration[1]])

allowed_zones = np.array(allowed_zones)

# Step 2: Generate LHS samples (normalized)
sampler = qmc.LatinHypercube(d=2)
lhs = sampler.random(n=num_samples)

# Step 3: Scale current directly
#currents = qmc.scale(lhs[:, 0], activation_current[0], activation_current[1])

scaled = qmc.scale(lhs, [activation_current[0], activation_duration[0]],
                         [activation_current[1], activation_duration[1]])

currents = scaled[:, 0]
durations_raw = scaled[:, 1]

# Step 4: Map duration samples into the allowed zones
zone_lengths = allowed_zones[:, 1] - allowed_zones[:, 0]
zone_probs = zone_lengths / np.sum(zone_lengths)
cum_probs = np.cumsum(zone_probs)

durations = []
for u in lhs[:, 1]:
    # Find which allowed zone this sample belongs to
    idx = np.searchsorted(cum_probs, u)
    idx = min(idx, len(allowed_zones) - 1)
    # Compute position within that zone
    if idx == 0:
        local_u = u / cum_probs[idx]
    else:
        local_u = (u - cum_probs[idx - 1]) / (cum_probs[idx] - cum_probs[idx - 1])
    val = allowed_zones[idx, 0] + local_u * (allowed_zones[idx, 1] - allowed_zones[idx, 0])
    durations.append(val)

durations = np.array(durations)

# Final combined samples
final_samples = np.column_stack((currents, durations))

print("Allowed zones for activation_duration:")
for z in allowed_zones:
    print(f"  {z[0]:.0f} – {z[1]:.0f}")

print("\nNew LHS Samples:")
for i, (c, d) in enumerate(final_samples):
    print(f"Sample {i+1}: current={c:.1f}, duration={d:.1f}")


Allowed zones for activation_duration:
  6112 – 24506
  31706 – 34319
  41519 – 52964
  65569 – 69029
  76229 – 86400

New LHS Samples:
Sample 1: current=1765.5, duration=7625.5
Sample 2: current=805.1, duration=43139.2
Sample 3: current=1883.8, duration=67639.6
Sample 4: current=1236.0, duration=17867.8
Sample 5: current=1408.1, duration=81894.6
Sample 6: current=1060.8, duration=46030.2
